[Rendered Notebook](http://nbviewer.jupyter.org/github/andersonfrailey/Notebook-Uploads/blob/master/newcpswts.ipynb)

In [1]:
import pandas as pd
import numpy as np
import taxcalc as tc
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, NumeralTickFormatter
from bokeh.layouts import layout, row, gridplot, column
from notebookfunctions import distribution, index_list, percentile
output_notebook()

Loading BokehJS ...

In [2]:
years = [i for i in range(2014, 2028)]

In [3]:
cps = pd.read_csv('cps_data/cps.csv.gz', compression='gzip')
new_wts = pd.read_csv('cps_stage2/cps_weights.csv.gz', compression='gzip')

In [4]:
new_wts.head()

,WT2014,WT2015,WT2016,WT2017,WT2018,WT2019,WT2020,WT2021,WT2022,WT2023,WT2024,WT2025,WT2026,WT2027
0,23001,23002,23312,23627,23946,24269,24597,24929,25265,25606,25952,26302,26658,27017
1,19460,19460,19723,19989,20259,20532,20809,21090,21375,21664,21956,22253,22553,22857
2,24527,24528,24859,25194,25535,25879,26229,26583,26942,27305,27674,28047,28426,28810
3,17132,17133,17364,17598,17836,18077,18321,18568,18819,19073,19330,19591,19856,20124
4,26012,26012,26363,26719,27080,27446,27816,28192,28572,28958,29349,29745,30147,30554


In [5]:
base = tc.Calculator(records=tc.Records.cps_constructor(),
                     policy=tc.Policy())
new = tc.Calculator(records=tc.Records(data=cps, weights=new_wts, start_year=2014),
                    policy=tc.Policy())

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [6]:
# collect data for each of the years
base_data = {}
new_data = {}
var_list = ['s006', 'iitax', 'combined', 'payrolltax', 'c00100',
            'expanded_income']
for year in years:
    base.advance_to_year(year)
    base.calc_all()
    new.advance_to_year(year)
    new.calc_all()
    base_data[year] = base.dataframe(var_list)
    new_data[year] = new.dataframe(var_list)

In [7]:
# tax liability by year in 2014
data = {'year': [], 'combined_base': [], 'combined_new': [], 'combined_change': [],
        'combined_pct_change': [], 'iitax_pct_change': [], 'payroll_pct_change': [],
        'iitax_base': [], 'iitax_new': [], 'iitax_change': [],
        'payroll_base': [], 'payroll_new': [], 'payroll_change': []}
for year in years:
    data['year'].append(year)
    combined_change = ((new_data[year]['combined'] - base_data[year]['combined']) *
                       base_data[year]['s006']).sum()
    iitax_change = ((new_data[year]['iitax'] - base_data[year]['iitax']) *
                    base_data[year]['s006']).sum()
    payroll_change = ((new_data[year]['payrolltax'] - base_data[year]['payrolltax']) *
                       base_data[year]['s006']).sum()
    data['combined_base'].append((base_data[year]['combined'] * base_data[year]['s006']).sum())
    data['combined_new'].append((new_data[year]['combined'] * new_data[year]['s006']).sum())
    data['combined_change'].append(combined_change)
    data['combined_pct_change'].append((combined_change / (base_data[year]['combined'] * base_data[year]['s006']).sum()) * 100)
    data['iitax_base'].append((base_data[year]['iitax'] * base_data[year]['s006']).sum())
    data['iitax_new'].append((new_data[year]['iitax'] * new_data[year]['s006']).sum())
    data['iitax_change'].append(iitax_change)
    data['iitax_pct_change'].append((iitax_change / (base_data[year]['iitax'] * base_data[year]['s006']).sum()) * 100)
    data['payroll_base'].append((base_data[year]['payrolltax'] * base_data[year]['s006']).sum())
    data['payroll_new'].append((new_data[year]['payrolltax'] * new_data[year]['s006']).sum())
    data['payroll_change'].append(payroll_change)
    data['payroll_pct_change'].append((payroll_change / (base_data[year]['payrolltax'] * base_data[year]['s006']).sum()) * 100)
cds = ColumnDataSource(data)
data_df = pd.DataFrame(data, index=data['year'])

In [8]:
f = figure(title='Tax Liability by Year')
f.line(x='year', y='combined_base', line_width=2, color='blue', source=cds, legend='Combined-Base')
f.line(x='year', y='combined_new', line_width=2, color='lightblue', source=cds, legend='Combined-new')
f.line(x='year', y='iitax_base', line_width=2, color='green', source=cds, legend='IITax-Base')
f.line(x='year', y='iitax_new', line_width=2, color='lime', source=cds, legend='IITax-New')
f.line(x='year', y='payroll_base', line_width=2, color='red', source=cds, legend='Payroll-Base')
f.line(x='year', y='payroll_new', line_width=2, color='pink', source=cds, legend='Payroll-New')
f.legend.location='top_left'
f.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')
f.xaxis[0].ticker.desired_num_ticks = 12
f.xaxis.minor_tick_line_color = None
show(f)

In [9]:
pd.options.display.float_format = '{:,.2f}'.format
print('Combined Tax Liability')
data_df[['combined_base', 'combined_new', 'combined_change', 'combined_pct_change']].transpose()

Combined Tax Liability


,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027
combined_base,"2,049,761,794,852.92","2,110,841,250,755.81","2,192,101,336,640.05","2,328,589,406,483.97","2,326,852,347,902.63","2,455,208,000,725.77","2,578,843,260,672.17","2,711,805,209,706.87","2,853,027,976,383.55","3,001,258,288,406.50","3,155,871,943,780.05","3,317,717,564,356.19","3,682,346,347,868.92","3,862,491,374,617.44"
combined_new,"2,084,752,076,132.89","2,137,874,933,442.45","2,219,643,951,652.22","2,357,191,746,986.31","2,363,959,739,967.22","2,493,924,513,016.17","2,619,372,887,105.36","2,753,951,759,800.80","2,897,062,235,232.81","3,047,148,484,816.51","3,203,649,724,810.17","3,367,417,517,237.28","3,724,762,661,228.35","3,906,357,652,304.50"
combined_change,"26,413,499,785.21","27,033,682,686.64","27,542,615,012.17","28,602,340,502.34","37,107,392,064.59","38,716,512,290.40","40,529,626,433.19","42,146,550,093.93","44,034,258,849.26","45,890,196,410.00","47,777,781,030.12","49,699,952,881.09","42,416,313,359.43","43,866,277,687.06"
combined_pct_change,1.29,1.28,1.26,1.23,1.59,1.58,1.57,1.55,1.54,1.53,1.51,1.50,1.15,1.14


In [10]:
print('Income Tax Liability')
data_df[['iitax_base', 'iitax_new', 'iitax_change', 'iitax_pct_change']].transpose()

Income Tax Liability


,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027
iitax_base,"1,052,768,062,368.58","1,079,321,792,276.94","1,118,073,598,335.02","1,194,682,943,874.67","1,134,225,986,055.48","1,207,623,350,219.63","1,278,600,912,769.22","1,355,456,344,576.62","1,435,537,108,564.83","1,518,783,771,055.15","1,605,135,131,486.43","1,695,298,234,281.98","1,984,937,041,617.15","2,086,678,633,847.68"
iitax_new,"1,085,393,273,929.21","1,106,352,032,276.81","1,145,612,801,798.59","1,223,281,486,997.70","1,171,329,397,011.78","1,246,335,693,620.15","1,319,126,187,111.95","1,397,598,372,680.23","1,479,566,644,433.17","1,564,669,015,699.37","1,652,907,725,317.07","1,744,992,747,382.13","2,027,347,646,137.48","2,130,538,949,290.70"
iitax_change,"26,410,123,218.17","27,030,239,999.87","27,539,203,463.57","28,598,543,123.03","37,103,410,956.30","38,712,343,400.52","40,525,274,342.73","42,142,028,103.61","44,029,535,868.34","45,885,244,644.23","47,772,593,830.65","49,694,513,100.16","42,410,604,520.34","43,860,315,443.03"
iitax_pct_change,2.51,2.50,2.46,2.39,3.27,3.21,3.17,3.11,3.07,3.02,2.98,2.93,2.14,2.10


In [11]:
print('Payroll Tax Liability')
data_df[['payroll_base', 'payroll_new', 'payroll_change', 'payroll_pct_change']].transpose()

Payroll Tax Liability


,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027
payroll_base,"996,993,732,484.34","1,031,519,458,478.87","1,074,027,738,305.03","1,133,906,462,609.30","1,192,626,361,847.16","1,247,584,650,506.14","1,300,242,347,902.95","1,356,348,865,130.26","1,417,490,867,818.72","1,482,474,517,351.36","1,550,736,812,293.63","1,622,419,330,074.21","1,697,409,306,251.77","1,775,812,740,769.76"
payroll_new,"999,358,802,203.69","1,031,522,901,165.65","1,074,031,149,853.63","1,133,910,259,988.61","1,192,630,342,955.44","1,247,588,819,396.02","1,300,246,699,993.41","1,356,353,387,120.57","1,417,495,590,799.64","1,482,479,469,117.14","1,550,741,999,493.10","1,622,424,769,855.15","1,697,415,015,090.87","1,775,818,703,013.80"
payroll_change,"3,376,567.03","3,442,686.77","3,411,548.60","3,797,379.31","3,981,108.28","4,168,889.89","4,352,090.46","4,521,990.32","4,722,980.93","4,951,765.78","5,187,199.47","5,439,780.93","5,708,839.09","5,962,244.04"
payroll_pct_change,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [12]:
def hist_plots(base_data, new_data, year):
    change_wts = new_data['s006'] - base_data['s006']
    pct_change_wts = (change_wts / base_data['s006'])
    hist_diff, edges_diff = np.histogram(change_wts, density=True, bins=50)
    hist_pct, edges_pct = np.histogram(pct_change_wts, density=True, bins=50)
    
    # create plots
    f1 = figure(title=f'Change in Weights for {year}', width=350, height=350)  # difference
    f1.quad(top=hist_diff, bottom=0, left=edges_diff[:-1], right=edges_diff[1:],
            fill_color='cyan', line_color='black')
    f1.yaxis.axis_label = 'Frequency'
    f1.xaxis.axis_label = 'Change'

    f2 = figure(title=f'Pct Change in Weights for {year}', width=350, height=350)  # pct change
    f2.quad(top=hist_pct, bottom=0, left=edges_pct[:-1], right=edges_pct[1:],
            fill_color='cyan', line_color='black')
    f2.yaxis.axis_label = 'Frequency'
    f2.xaxis.axis_label = 'Pct Change'
    
    return f1, f2

In [13]:
# analyze change in each record's weight
plot_list_diff = []  # holds graphs showing difference in wt
plot_list_pct = []  # holds graphs showing histogram of pct change in wt
for year in years:
    base_data1 = base_data[year]
    new_data1 = new_data[year]

    hist_diff1, hist_pct1 = hist_plots(base_data1, new_data1, year)
    plot_list_diff.append(hist_diff1)
    plot_list_pct.append(hist_pct1)

In [14]:
show(column(plot_list_diff))

In [15]:
show(column(plot_list_pct))